In [2]:
# install the necessary libraries
!pip install requests
!pip install bs4
!pip install lxml

In [3]:
# import the packages 

import pandas as pd
import requests

from bs4 import BeautifulSoup

In [5]:
# specify the url
# create a variable 'page' and use request.get to get that page
# response code 200 mean everything is fine and we have a connection

url = 'https://www.worldometers.info/world-population/population-by-country/'

page = requests.get(url)
page

<Response [200]>

In [8]:
# 1. next step is to gather the html code from that website
# 2. create a if block to make sure that we only get the html code if everything correct
# 3. define the html doc as a text
# 4. look at the html and pull out the information we need, create a soup object
# 5. print using prettify()
if page.status_code ==200:
    html_doc = page.text
    
soup = BeautifulSoup(html_doc)

print(soup.prettify())
    

<!DOCTYPE html>
<!--[if IE 8]> <html lang="en" class="ie8"> <![endif]-->
<!--[if IE 9]> <html lang="en" class="ie9"> <![endif]-->
<!--[if !IE]><!-->
<html lang="en">
 <!--<![endif]-->
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <title>
   Population by Country (2022) - Worldometer
  </title>
  <meta content="List of countries and dependencies in the world ranked by population, from the most populated. Growth rate, median age, fertility rate, area, density, population density, urbanization, urban population, share of world population." name="description"/>
  <link href="/favicon/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
  <link href="/favicon/apple-icon-57x57.png" rel="apple-touch-icon" sizes="57x57"/>
  <link href="/favicon/apple-icon-60x60.png" rel="apple-touch-icon" sizes="60x60"/>
  <link href="/favicon/apple-icon-72x72.png" rel="apple-touch-icon" siz

# Use insepect on the browser to find the table

• go to the webpage on the browser, right click and select 'inspect'.

• Find the div that contains the table and find the table id. In this case the table id is example2

In [11]:
# Now we want to extract the table from this url

table = soup.find('table', attrs={'id':'example2'})
print(table.prettify())

<table cellspacing="0" class="table table-striped table-bordered" id="example2" width="100%">
 <thead>
  <tr>
   <th>
    #
   </th>
   <th>
    Country (or dependency)
   </th>
   <th>
    Population
    <br/>
    (2020)
   </th>
   <th>
    Yearly
    <br/>
    Change
   </th>
   <th>
    Net
    <br/>
    Change
   </th>
   <th>
    Density
    <br/>
    (P/Km²)
   </th>
   <th>
    Land Area
    <br/>
    (Km²)
   </th>
   <th>
    Migrants
    <br/>
    (net)
   </th>
   <th>
    Fert.
    <br/>
    Rate
   </th>
   <th>
    Med.
    <br/>
    Age
   </th>
   <th>
    Urban
    <br/>
    Pop %
   </th>
   <th>
    World
    <br/>
    Share
   </th>
  </tr>
 </thead>
 <tbody>
  <tr>
   <td>
    1
   </td>
   <td style="font-weight: bold; font-size:15px; text-align:left">
    <a href="/world-population/china-population/">
     China
    </a>
   </td>
   <td style="font-weight: bold;">
    1,439,323,776
   </td>
   <td>
    0.39 %
   </td>
   <td>
    5,540,090
   </td>
   <td>
    1

As you can see from the output above table rows are represented by tr and individual data in the rows is represented by td.

Now we need to extract a list of rows

In [12]:
# Find all the rows in the table

rows = table.find_all('tr')
rows


[<tr> <th>#</th> <th>Country (or dependency)</th> <th>Population<br/> (2020)</th> <th>Yearly<br/> Change</th> <th>Net<br/> Change</th> <th>Density<br/> (P/Km²)</th> <th>Land Area<br/> (Km²)</th> <th>Migrants<br/> (net)</th> <th>Fert.<br/> Rate</th> <th>Med.<br/> Age</th> <th>Urban<br/> Pop %</th> <th>World<br/> Share</th> </tr>,
 <tr> <td>1</td> <td style="font-weight: bold; font-size:15px; text-align:left"><a href="/world-population/china-population/">China</a></td> <td style="font-weight: bold;">1,439,323,776</td> <td>0.39 %</td> <td>5,540,090</td> <td>153</td> <td>9,388,211</td> <td>-348,399</td> <td>1.7</td> <td>38</td> <td>61 %</td> <td>18.47 %</td> </tr>,
 <tr> <td>2</td> <td style="font-weight: bold; font-size:15px; text-align:left"><a href="/world-population/india-population/">India</a></td> <td style="font-weight: bold;">1,380,004,385</td> <td>0.99 %</td> <td>13,586,631</td> <td>464</td> <td>2,973,190</td> <td>-532,687</td> <td>2.2</td> <td>28</td> <td>35 %</td> <td>17.70 %</t

# Iterate through each of the rows and each of the values in those rows.

Each time we go through a row we have to save the values and value types we have scraped at each row level.

we do that by using for loops

In [14]:

# create a variable where we can store the extracted values
output = []

# make a list of column names so we can match the extracted row values to columns
column_names =['ID','Country (or dependency)', 'Population (2020)',
              'Yearly Change', 'Net Change', 'Density (p/km2)',
              'LAND AREA (Km2)', 'Migrants (net)', 'Fert. Rate',
              'Med. Age', 'Urbn Pop', 'World Share']

# Use a for loop to extract values
# each row represents a country
for country in rows: #-- define country
    country_data = country.find_all('td') #-- for the particular 'td' we are iterating it will find all
    if country_data: #-- use if statement to make sure we are not iterating from an empty variable
        country_text = [td.text for td in country_data] #-- if so we only get the date inside the td eg. <td> 500 </td> as text. use list []
        output.append(dict(zip(column_names, country_text))) # we then append to a list using appen(dect(zip)) this will put the column names and the extracted country text together
        
output
    

[{'ID': '1',
  'Country (or dependency)': 'China',
  'Population (2020)': '1,439,323,776',
  'Yearly Change': '0.39 %',
  'Net Change': '5,540,090',
  'Density (p/km2)': '153',
  'LAND AREA (Km2)': '9,388,211',
  'Migrants (net)': '-348,399',
  'Fert. Rate': '1.7',
  'Med. Age': '38',
  'Urbn Pop': '61 %',
  'World Share': '18.47 %'},
 {'ID': '2',
  'Country (or dependency)': 'India',
  'Population (2020)': '1,380,004,385',
  'Yearly Change': '0.99 %',
  'Net Change': '13,586,631',
  'Density (p/km2)': '464',
  'LAND AREA (Km2)': '2,973,190',
  'Migrants (net)': '-532,687',
  'Fert. Rate': '2.2',
  'Med. Age': '28',
  'Urbn Pop': '35 %',
  'World Share': '17.70 %'},
 {'ID': '3',
  'Country (or dependency)': 'United States',
  'Population (2020)': '331,002,651',
  'Yearly Change': '0.59 %',
  'Net Change': '1,937,734',
  'Density (p/km2)': '36',
  'LAND AREA (Km2)': '9,147,420',
  'Migrants (net)': '954,806',
  'Fert. Rate': '1.8',
  'Med. Age': '38',
  'Urbn Pop': '83 %',
  'World Shar

In [18]:
data = pd.DataFrame(output)
data.head()

data.to_csv("countries.csv", index=False)



In [19]:
import json

output_json = json.dumps(output)

In [20]:
output_json

'[{"ID": "1", "Country (or dependency)": "China", "Population (2020)": "1,439,323,776", "Yearly Change": "0.39 %", "Net Change": "5,540,090", "Density (p/km2)": "153", "LAND AREA (Km2)": "9,388,211", "Migrants (net)": "-348,399", "Fert. Rate": "1.7", "Med. Age": "38", "Urbn Pop": "61 %", "World Share": "18.47 %"}, {"ID": "2", "Country (or dependency)": "India", "Population (2020)": "1,380,004,385", "Yearly Change": "0.99 %", "Net Change": "13,586,631", "Density (p/km2)": "464", "LAND AREA (Km2)": "2,973,190", "Migrants (net)": "-532,687", "Fert. Rate": "2.2", "Med. Age": "28", "Urbn Pop": "35 %", "World Share": "17.70 %"}, {"ID": "3", "Country (or dependency)": "United States", "Population (2020)": "331,002,651", "Yearly Change": "0.59 %", "Net Change": "1,937,734", "Density (p/km2)": "36", "LAND AREA (Km2)": "9,147,420", "Migrants (net)": "954,806", "Fert. Rate": "1.8", "Med. Age": "38", "Urbn Pop": "83 %", "World Share": "4.25 %"}, {"ID": "4", "Country (or dependency)": "Indonesia", 

In [21]:
with open('countries.json', 'w') as f:
    json.dump(output, f)


In [22]:
data = pd.read_csv('countries.csv')
data.head()

ID Country (or dependency) Population (2020) Yearly Change  Net Change  \
0   1                   China     1,439,323,776        0.39 %   5,540,090   
1   2                   India     1,380,004,385        0.99 %  13,586,631   
2   3           United States       331,002,651        0.59 %   1,937,734   
3   4               Indonesia       273,523,615        1.07 %   2,898,047   
4   5                Pakistan       220,892,340        2.00 %   4,327,022   

  Density (p/km2) LAND AREA (Km2) Migrants (net) Fert. Rate Med. Age Urbn Pop  \
0             153       9,388,211       -348,399        1.7       38     61 %   
1             464       2,973,190       -532,687        2.2       28     35 %   
2              36       9,147,420        954,806        1.8       38     83 %   
3             151       1,811,570        -98,955        2.3       30     56 %   
4             287         770,880       -233,379        3.6       23     35 %   

  World Share  
0     18.47 %  
1     17.70 %  
2      4.25 %  
3      3.51 %  
4      2.83 %

In [23]:
data = pd.read_json ('countries.json')

data.head()

ID Country (or dependency) Population (2020) Yearly Change  Net Change  \
0   1                   China     1,439,323,776        0.39 %   5,540,090   
1   2                   India     1,380,004,385        0.99 %  13,586,631   
2   3           United States       331,002,651        0.59 %   1,937,734   
3   4               Indonesia       273,523,615        1.07 %   2,898,047   
4   5                Pakistan       220,892,340        2.00 %   4,327,022   

  Density (p/km2) LAND AREA (Km2) Migrants (net) Fert. Rate Med. Age Urbn Pop  \
0             153       9,388,211       -348,399        1.7       38     61 %   
1             464       2,973,190       -532,687        2.2       28     35 %   
2              36       9,147,420        954,806        1.8       38     83 %   
3             151       1,811,570        -98,955        2.3       30     56 %   
4             287         770,880       -233,379        3.6       23     35 %   

  World Share  
0     18.47 %  
1     17.70 %  
2      4.25 %  
3      3.51 %  
4      2.83 %